<a href="https://colab.research.google.com/github/Saurav-exe/IMPACTX/blob/main/Model_Building.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/IMPACTX/preprocessed_data.csv')
display(df.head())

,StateDistrictMarket,Commodity,Variety,Grade,Min Price,Max Price,Modal Price,Freshness
0,90,22,57,0,4100.0,4500.0,4350.0,6
1,90,28,311,0,2200.0,3000.0,2450.0,6
2,90,32,81,0,2350.0,3000.0,2700.0,6
3,90,40,87,0,7000.0,7500.0,7250.0,6
4,90,60,112,0,8400.0,9000.0,8850.0,6


In [4]:
features = ['StateDistrictMarket', 'Commodity', 'Variety', 'Grade', 'Min Price', 'Max Price', 'Freshness']
X = df[features]
y = df['Modal Price']


In [5]:
for column in ['Min Price', 'Max Price']:
    Q1 = X[column].quantile(0.25)
    Q3 = X[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Remove outliers
    X = X[(X[column] >= lower_bound) & (X[column] <= upper_bound)]
    y = y[(X.index)]


# Check the data again
display(X.describe())
display(y.describe())

,StateDistrictMarket,Commodity,Variety,Grade,Min Price,Max Price,Freshness
count,21186.000000,21186.000000,21186.000000,21186.000000,21186.000000,21186.000000,21186.000000
mean,703.386718,102.387614,236.677617,0.202492,3188.089502,3735.732897,2.689890
std,389.416404,70.494930,112.629813,0.595844,2110.745071,2293.305338,1.666346
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,339.000000,28.000000,125.000000,0.000000,1600.000000,2000.000000,1.000000
50%,713.000000,92.000000,311.000000,0.000000,2500.000000,3000.000000,2.000000
75%,1091.000000,173.000000,311.000000,0.000000,4500.000000,5025.000000,4.000000
max,1300.000000,233.000000,451.000000,3.000000,9875.000000,10355.000000,6.000000


,Modal Price
count,21186.000000
mean,3480.437808
std,2185.157044
min,0.830000
25%,1850.000000
50%,2762.500000
75%,4855.000000
max,16500.000000


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


###Using Min-Price.

In [17]:

X_train_simple = X_train[['Min Price']]


model_simple = LinearRegression()

model_simple.fit(X_train_simple, y_train)

X_test_simple = X_test[['Min Price']]

y_pred_simple = model_simple.predict(X_test_simple)

#Model-2: Multiple Linear Regression(MLR)

In [18]:
model_multiple = LinearRegression()
model_multiple.fit(X_train, y_train)

y_pred_multiple = model_multiple.predict(X_test)

#Model-3: random Forest regressor (RFR)

In [19]:
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

rf_model.fit(X_train, y_train)

y_pred_rf = rf_model.predict(X_test)

#Model Evaluation.

In [21]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [23]:
def evaluate_model(y_true, y_pred, model_name):

    mse = mean_squared_error(y_true, y_pred)
    rmse = mse**0.5  # Root Mean Squared Error
    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)

    print(f"Model: {model_name}")
    print(f"Mean Squared Error (MSE): {mse:.5f}")
    print(f"Root Mean Squared Error (RMSE): {rmse:.5f}")
    print(f"R-squared (R2): {r2:.5f}")
    print(f"Mean Absolute Error (MAE): {mae:.5f}")
    print("-" * 30)

# Evaluate each model
evaluate_model(y_test, y_pred_simple, "Simple Linear Regression")
evaluate_model(y_test, y_pred_multiple, "Multiple Linear Regression")
evaluate_model(y_test, y_pred_rf, "Random Forest Regressor")

Model: Simple Linear Regression
Mean Squared Error (MSE): 223720.95369
Root Mean Squared Error (RMSE): 472.99149
R-squared (R2): 0.95343
Mean Absolute Error (MAE): 283.63154
------------------------------
Model: Multiple Linear Regression
Mean Squared Error (MSE): 37806.34633
Root Mean Squared Error (RMSE): 194.43854
R-squared (R2): 0.99213
Mean Absolute Error (MAE): 73.23217
------------------------------
Model: Random Forest Regressor
Mean Squared Error (MSE): 24025.89864
Root Mean Squared Error (RMSE): 155.00290
R-squared (R2): 0.99500
Mean Absolute Error (MAE): 59.78992
------------------------------
